# Reward Model

En este notebook tenemos como objetivo ver cómo funciona un modelo de recompensa, cargarlo y hacer algunas pruebas


## Instalación de las dependencias


In [2]:
!pip install transformers torch accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

Instanciemos el modelo, el tokenizer y hagamos print de las etiquetas.

Usaremos [facebook/roberta-hate-speech-dynabench-r4-target](https://huggingface.co/facebook/roberta-hate-speech-dynabench-r4-target)

En este modelo, 'hate' se define como "discurso abusivo que tiene como objetivo características específicas de un grupo, como el origen étnico, la religión, el género o la orientación sexual".


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

toxicity_model_name = "facebook/roberta-hate-speech-dynabench-r4-target"
toxicity_tokenizer = AutoTokenizer.from_pretrained(toxicity_model_name)

toxicity_model = AutoModelForSequenceClassification.from_pretrained(toxicity_model_name, device_map="auto")

print(toxicity_model.config.id2label)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/816 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

{0: 'nothate', 1: 'hate'}


In [4]:
def evaluate_toxicity(prompt):
  device = next(toxicity_model.parameters()).device

  toxicity_input_ids = toxicity_tokenizer(prompt, return_tensors="pt").input_ids.to(device)

  not_hate_index = 0
  logits = toxicity_model(input_ids=toxicity_input_ids).logits
  probabilities = logits.softmax(dim=-1).tolist()[0]
  nothate_reward = (logits[:, not_hate_index]).tolist()

  print(f'logits [not hate, hate]: {logits.tolist()[0]}')
  print(f'probabilities [not hate, hate]: {probabilities}')
  print(f'reward (high): {nothate_reward}')


In [5]:
prompts = [
    "#Person 1# You are a shit person and I hate you",
    "#Person 1# You are a shit person and I like you",
    "#Person 1# You are a wonderful person and I hate you",
    "#Person 1# You are a wonderful person and I like you",
    "Angular te ofrece control total, eficiencia y escalabilidad para proyectos serios, pero tiene una curva de aprendizaje pronunciada.",
    "Angular es un framework de JavaScript poco útil, es mejor React."
]

for prompt in prompts:
    evaluate_toxicity(prompt)
    print("\n---\n")

logits [not hate, hate]: [-2.1874988079071045, 1.9269628524780273]
probabilities [not hate, hate]: [0.016072191298007965, 0.9839277863502502]
reward (high): [-2.1874988079071045]

---

logits [not hate, hate]: [-1.062474250793457, 0.7645438313484192]
probabilities [not hate, hate]: [0.1385939121246338, 0.861406147480011]
reward (high): [-1.062474250793457]

---

logits [not hate, hate]: [0.4744216203689575, -0.49220210313796997]
probabilities [not hate, hate]: [0.7244459986686707, 0.27555394172668457]
reward (high): [0.4744216203689575]

---

logits [not hate, hate]: [4.589840888977051, -4.067539215087891]
probabilities [not hate, hate]: [0.9998262524604797, 0.00017380894860252738]
reward (high): [4.589840888977051]

---

logits [not hate, hate]: [3.8288633823394775, -3.2355377674102783]
probabilities [not hate, hate]: [0.9991457462310791, 0.0008542764117009938]
reward (high): [3.8288633823394775]

---

logits [not hate, hate]: [4.447649002075195, -3.90937876701355]
probabilities [not 